Esta cosa debería de tomar dos archivos, una sucesión de tiempos del GD y otra del CA, y checar coincidencias. Usemos Julia solamente y tratemos de paralelizar.

In [58]:
using JLD

In [69]:
ListaCA=readdir("./MuchasEspigas/CA3/");
ListaGD=readdir("./MuchasEspigas/DG/");

In [63]:
EntradasYValores=Dict{Any,Any}()
Randomizados=Dict{Any,Any}()

Dict{Any,Any} with 0 entries

In [3]:
ejemploCA=readdlm("./DatosEnero2016/CA3/Ch31_46.txt");
ejemploGD=readdlm("./DatosEnero2016/DG/Ch24_27.txt");

In [7]:
function CoincidenGDaCA(GD::Array, CA::Array)
    cuantosCA=length(CA)
    cuantosGD=length(GD)
    #Unidades en frames, son 1/7022 de segundo
    desde=13
    hasta=71
    coinciden::Int64=0
    

    j=1
    
    while j<cuantosGD
       
        k=1

        while k<cuantosCA
            dift=(CA[k]-GD[j])           
            if desde<dift<hasta
                coinciden+=1
            elseif dift>hasta
                break
            end
            k+=1        
        end
        j+=1       
    end
    
    return coinciden
end

CoincidenGDaCA (generic function with 1 method)

In [8]:
CoincidenGDaCA(ejemploGD,ejemploCA)

1

In [70]:
CoincidenciasCausales=Array(Int, 1,7)
Pupu=Int32[]
laregexp=r"([0-9]{2})_([0-9]{2})"
@time for k in ListaCA, j in ListaGD
    ejemploCA=readdlm("./MuchasEspigas/CA3/$k")
    ejemploGD=readdlm("./MuchasEspigas/DG/$j")  
    
    parapa=match(laregexp, k)
    perepe=match(laregexp,j)
    #println(typeof(parapa), parapa,k,j)
    
    parteCA=[parse(Int64,s) for s=parapa.captures ]  
    parteGD=[parse(Int64,z) for z=perepe.captures ]
    
    cuantos=CoincidenGDaCA(ejemploGD,ejemploCA)
    
    Pupu=[parteGD; parteCA; [cuantos; length(ejemploGD); length(ejemploCA)]]'
    CoincidenciasCausales=vcat(CoincidenciasCausales, Pupu)
    
end

CoincidenciasCausales=CoincidenciasCausales[2:end,:];


 61.568507 seconds (169.84 M allocations: 42.745 GB, 5.57% gc time)


In [71]:
writedlm("MuchasEspigas/CoincidenciasCausales01.dat", CoincidenciasCausales);

In [72]:
extrema(CoincidenciasCausales[:,5])

(0,523)

In [73]:
function sacaintervalos(xxs::Array)
    result=zeros(xxs)
    result[1]=xxs[1]
    for j=2:length(xxs)
        result[j]=xxs[j]-xxs[j-1]
    end
    return result
end

function tiempodeintervalos(xxs::Array)
    largo=length(xxs)
    result=zeros(largo)
    result[1]=xxs[1]
    for j=1:largo-1
        result[j+1]=result[j]+xxs[j+1]
    end
    return result
end
    

tiempodeintervalos (generic function with 1 method)

In [ ]:
RandomCoincide=zeros(Float32, 1,7)
Pupu=Float32[]
cuantasveces=1000
promcoincide=0
HistoGramasTotales=Dict{Array{Int}, Array}()
@time for k in ListaCA, j in ListaGD
    
    ejemploCA=readdlm("./MuchasEspigas/CA3/$k")
    ejemploGD=readdlm("./MuchasEspigas/DG/$j")
    
    parapa=match(laregexp,k)
    perepe=match(laregexp,j)
    
    parteCA=[parse(Int64,s) for s=parapa.captures ]
    parteGD=[parse(Int64,s) for s=perepe.captures ]
        
    intervalosGD=sacaintervalos(vec(ejemploGD))
    intervalosCA=sacaintervalos(vec(ejemploCA))
    
    HistoGramasTotales[[parteGD ; parteCA]]=zeros(Int,50)
    
    for t in 1:cuantasveces
        shuffle!(intervalosGD)
        shuffle!(intervalosCA)
        novoCA=tiempodeintervalos(intervalosCA)
        novoGD=tiempodeintervalos(intervalosGD)
        
        coincidencias=CoincidenGDaCA(novoGD, novoCA)
        if coincidencias<50
        HistoGramasTotales[[parteGD ; parteCA]][coincidencias+1]+=1
        end
        promcoincide+=coincidencias
        
    end
    
    promcoincide/=cuantasveces
    Pupu=[parteGD; parteCA; [promcoincide; length(ejemploGD); length(ejemploCA)]]'
    RandomCoincide=vcat(RandomCoincide, Pupu)

end
RandomCoincide=RandomCoincide[2:end]

writedlm("RevueltasCoincidenPureJulia01.dat", RandomCoincide)

In [56]:
save("MuchasEspigas/HistogramasMuchasEspigasShuffle01.jld", "HistogramasTotales", HistoGramasTotales)